In [5]:
import requests
import datetime
import sqlite3

API_KEY = "your_api_key_here"
CITY = "Bonn"

# Fetch current weather data from the API
url = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={CITY}&aqi=no"
response = requests.get(url)
data = response.json()

if "current" in data:
    weather_condition = data["current"]["condition"]["text"]
    temp = data["current"]["temp_c"]
    print(f"Weather: {weather_condition}, Temperature: {temp}°C")
else:
    print("Weather data unavailable. Check API response.")

# Determine time of day
hour = datetime.datetime.now().hour
if 6 <= hour < 12:
    time_of_day = "Morning"
elif 12 <= hour < 18:
    time_of_day = "Daytime"
else:
    time_of_day = "Night"

print("Time of Day:", time_of_day)

# Normalize "Clear" to "Sunny"
def normalize_weather_condition(weather):
    if weather in ["Clear", "Sunny"]:
        return "Sunny"
    return weather

# Modify mood determination based on weather, temperature, and time of day
def determine_mood(weather, temp, time_of_day):
    weather = normalize_weather_condition(weather)  # Normalize the weather condition

    temp_category = "Warm" if temp > 15 else "Cold"
    
    mood_map = {
        "Sunny": {
            "Warm": {"Morning": "Energetic", "Daytime": "Energetic", "Night": "Relaxed"},
            "Cold": {"Morning": "Happy", "Daytime": "Happy", "Night": "Happy"}
        },
        "Rain": {
            "Any": {"Morning": "Calm", "Daytime": "Calm", "Night": "Calm"}
        },
        "Snow": {
            "Any": {"Morning": "Neutral", "Daytime": "Neutral", "Night": "Relaxed"}
        },
        "Cloudy": {
            "Any": {"Morning": "Neutral", "Daytime": "Neutral", "Night": "Energetic"}
        },
        "Thunderstorm": {
            "Any": {"Morning": "Energetic", "Daytime": "Energetic", "Night": "Energetic"}
        }
    }

    # Default to "Neutral" if no match found
    return mood_map.get(weather, {}).get(temp_category, {}).get(time_of_day, "Neutral")

# Predict mood based on the current weather and time of day
mood = determine_mood(weather_condition, temp, time_of_day)
print("Predicted Mood:", mood)

# Predicting moods based on Valence and energy ranges
mood_ranges = {
    "Happy": (0.7, 1.0, 0.6, 1.0),
    "Energetic": (0.6, 1.0, 0.8, 1.0),
    "Calm": (0.2, 0.5, 0.1, 0.4),
    "Relaxed": (0.4, 0.7, 0.2, 0.5),
    "Neutral": (0.4, 0.6, 0.4, 0.7)
}

# Connect to the music recommendation database
conn = sqlite3.connect("music_recommendation.db")
cursor = conn.cursor()

# Insert current weather data into the weather table
cursor.execute("""
    INSERT INTO weather (city, weather_condition, temperature, time_of_day, mood)
    VALUES (?, ?, ?, ?, ?)
""", (CITY, weather_condition, temp, time_of_day, mood))
conn.commit()
conn.close()

# Use the latest weather data to recommend songs
conn_spotify = sqlite3.connect("Spotify.db")
cursor_spotify = conn_spotify.cursor()

# Get the predicted mood from the weather data
cursor_spotify.execute("""
    SELECT track_name, artists, track_genre, valence, energy
    FROM songs_cleaned
    WHERE valence BETWEEN ? AND ?
    AND energy BETWEEN ? AND ?
    ORDER BY RANDOM()
    LIMIT 5
""", mood_ranges[mood])

'''
cursor_spotify.execute("""
    SELECT track_name, artists, track_genre, valence, energy
    FROM songs_cleaned
    WHERE valence BETWEEN ? AND ?
    AND energy BETWEEN ? AND ?
    LIMIT 5
""", mood_ranges[mood])
'''

recommended_songs = cursor_spotify.fetchall()

if recommended_songs:
    print(f"🎵 Recommended Songs for Mood: {mood}")
    for song in recommended_songs:
        print(f"{song[0]} - {song[1]} ({song[2]}) | Valence: {song[3]}, Energy: {song[4]}")
else:
    print(f"⚠️ No songs found for mood: {mood}. Check dataset.")

conn_spotify.close()

Weather: Clear, Temperature: 3.2°C
Time of Day: Night
Predicted Mood: Happy
🎵 Recommended Songs for Mood: Happy
All night alone - chris lake edit - Josement;chris lake (minimal-techno) | Valence: 0.848, Energy: 0.857
Dpkt - Roshan jamrock;yunohoo;mc sai;dopeadelicz;young ruff;chico (malay) | Valence: 0.741, Energy: 0.842
Hauskindbaum - Das lumpenpack (ska) | Valence: 0.821, Energy: 0.872
Botas sucias - Pappo (heavy-metal) | Valence: 0.889, Energy: 0.785
In the dark - Purple disco machine;sophie and the giants (deep-house) | Valence: 0.775, Energy: 0.77
